In [ ]:
import numpy as np
import pandas as pd
import random
import os
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
train_dir = '../input/sartorius-cell-instance-segmentation/train'

In [ ]:
df_train = pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv")
df_train

# Visualize Annotaion
referred to https://www.kaggle.com/ihelon/cell-segmentation-run-length-decoding

In [ ]:
def rle_decode(mask_rle, shape, color=1):
    s = mask_rle.split()
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)            
    for start, end in zip(starts, ends):
        img[start : end] = color    
    return img.reshape(shape)

In [ ]:
def plot_masks(image_id, colors=True):
    labels = df_train[df_train["id"] == image_id]["annotation"].tolist()    
    if colors:
        mask = np.zeros((520, 704, 3))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 3), color=np.random.rand(3))
    else:
        mask = np.zeros((520, 704, 1))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 1))
            
    mask = mask.clip(0, 1)
    image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 16))
    plt.subplot(3, 1, 1)
    plt.imshow(image)
    plt.axis("off")
    
    plt.subplot(3, 1, 2)
    plt.imshow(image)
    plt.imshow(mask, alpha=0.5)
    plt.axis("off")
    
    plt.subplot(3, 1, 3)
    plt.imshow(mask)
    plt.axis("off")
    
    plt.show();

In [ ]:
plot_masks("ffdb3cc02eef", colors=False)

In [ ]:
def plot_masks2(image_id, colors=True):
    labels = df_train[df_train["id"]==image_id]["annotation"].tolist()    
    if colors:
        mask = np.zeros((520,704,3))
        for label in labels:
            mask += rle_decode(label, shape=(520,704,3), color=np.random.rand(3))
    else:
        mask = np.zeros((520,704,1))
        for label in labels:
            mask += rle_decode(label, shape=(520,704,1))
            
    mask = mask.clip(0,1)
    return mask

In [ ]:
Name0=df_train['cell_type'].unique().tolist()
Name=sorted(Name0)

In [ ]:
N=list(range(len(Name)))
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
trainset=[]
trainname=[]
for im in tqdm(os.listdir(train_dir)):
    mask=plot_masks2(im[0:-4])
    celltype=df_train[df_train["id"]==im[0:-4]]['cell_type'].tolist()[0]
    mask2=cv2.resize(mask,dsize=(70,52),interpolation=cv2.INTER_CUBIC)
    trainset+=[mask2]
    trainname+=[celltype]

In [ ]:
m=len(trainset)
M=list(range(m))
random.seed(2021)
random.shuffle(M)

In [ ]:
fig, axs = plt.subplots(3,3,figsize=(15,15))
for i in range(9):
    r=i//3
    c=i%3
    img1=trainset[M[i]]
    name=trainname[M[i]]
    ax=axs[r][c].axis("off")
    ax=axs[r][c].set_title(name)
    ax=axs[r][c].imshow(img1)
plt.show()